In [1]:
from drugs import Drugs
import pandas as pd

In [2]:
df = pd.read_csv("../data/drugs_train.csv")

In [3]:
df_ingredient= pd.read_csv("../data/active_ingredients.csv")

In [4]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, random_state=40, test_size=0.2)

In [5]:
len(df_train), len(df_test)

(6851, 1713)

In [6]:
df_ingredient_train = df_ingredient[df_ingredient.drug_id.isin(df_train.drug_id)]
df_ingredient_val = df_ingredient[df_ingredient.drug_id.isin(df_test.drug_id)]

In [7]:
df_ingredient_train.drug_id.nunique(), df_ingredient_val.drug_id.nunique()

(6851, 1713)

In [8]:
drugs = Drugs()

In [11]:
drugs.fit(df=df_train, df_ingredient=df_ingredient_train, val_df=df_test, val_df_ingredient=df_ingredient_val)

[0]	validation_0-rmse:65.33997	validation_1-rmse:75.41024
[1]	validation_0-rmse:51.10884	validation_1-rmse:69.76710
[2]	validation_0-rmse:41.73246	validation_1-rmse:66.66173
[3]	validation_0-rmse:35.34375	validation_1-rmse:65.33466
[4]	validation_0-rmse:31.42314	validation_1-rmse:64.46288
[5]	validation_0-rmse:28.83915	validation_1-rmse:64.03953
[6]	validation_0-rmse:27.25506	validation_1-rmse:63.74276
[7]	validation_0-rmse:26.21535	validation_1-rmse:63.75871
[8]	validation_0-rmse:25.44148	validation_1-rmse:63.57790
[9]	validation_0-rmse:25.05099	validation_1-rmse:63.47045
[10]	validation_0-rmse:24.50150	validation_1-rmse:63.36183
[11]	validation_0-rmse:23.97293	validation_1-rmse:63.39420
[12]	validation_0-rmse:23.63574	validation_1-rmse:63.31099
[13]	validation_0-rmse:23.30477	validation_1-rmse:63.40172
[14]	validation_0-rmse:23.22497	validation_1-rmse:63.40575
[15]	validation_0-rmse:23.18578	validation_1-rmse:63.37044
[16]	validation_0-rmse:22.99711	validation_1-rmse:63.77304
[17]	va